In [1]:
# Import required packages
import argparse
import os.path as osp
import pickle
import os

import gym
import numpy as np
import h5py
import torch as th
import torch.nn as nn
from gym.wrappers import TimeLimit
from tqdm.notebook import tqdm

import mani_skill2.envs
from mani_skill2.utils.wrappers import RecordEpisode
from torch.utils.data import Dataset, DataLoader


In [2]:
env_id = "StackCube-v1"

In [3]:
# loads h5 data into memory for faster access

def tensor_to_numpy(x):
    # moves all tensors to numpy. This is just for SB3 as SB3 does not optimize for observations stored on the GPU.
    if th.is_tensor(x):
        return x.cpu().numpy()
    return x
def convert_observation(observation):
    # flattens the original observation by flattening the state dictionaries
    # and combining the rgb and depth images

    # image data is not scaled here and is kept as uint16 to save space
    image_obs = observation["image"]
    rgb = image_obs["base_camera"]["rgb"]
    depth = image_obs["base_camera"]["depth"]
    rgb2 = image_obs["hand_camera"]["rgb"]
    depth2 = image_obs["hand_camera"]["depth"]

    # we provide a simple tool to flatten dictionaries with state data
    from mani_skill2.utils.common import flatten_state_dict
    obs_ext = observation["extra"]["tcp_pose"]
    state = np.hstack(
        [
            flatten_state_dict(observation["agent"]),
            #obs_ext,
            flatten_state_dict(observation["extra"]),
        ]
    )
    #print(state.shape)
    # combine the RGB and depth images
    rgbd = np.concatenate([rgb, depth, rgb2, depth2], axis=-1)
    obs = dict(rgbd=rgbd, state=state)
    return obs
def rescale_rgbd(rgbd):
    # rescales rgbd data and changes them to floats
    rgb1 = rgbd[..., 0:3] / 255.0
    rgb2 = rgbd[..., 4:7] / 255.0
    depth1 = rgbd[..., 3:4] / (2**10)
    depth2 = rgbd[..., 7:8] / (2**10) 
    return np.concatenate([rgb1, depth1, rgb2, depth2], axis=-1)
class ManiSkill2Dataset(Dataset):
    def __init__(self, dataset_file: str, load_count=-1) -> None:
        self.dataset_file = dataset_file
        import pickle
        self.data = pickle.load(open(self.dataset_file, 'rb'))
        print('load in again')
        self.episodes = len(self.data.keys())

        self.obs_state = []
        self.obs_rgbd = []
        self.actions = []
        self.total_frames = 0
        if load_count == -1:
            load_count = self.episodes
        for eps_id in tqdm(range(load_count)):
            # eps = self.episodes[eps_id]
            trajectory = self.data[f"traj_{eps_id}"]

            # convert the original raw observation with our batch-aware function
            obs = convert_observation(trajectory["obs"])
            # we use :-1 to ignore the last obs as terminal observations are included
            # and they don't have actions
            self.obs_rgbd.append(obs['rgbd'][:-1])
            self.obs_state.append(obs['state'][:-1])
            self.actions.append(trajectory["actions"])
        #test = np.concatenate(self.obs_rgbd,axis=0)
        #print(f'shape of test{test.shape}')
        self.obs_rgbd = np.concatenate(self.obs_rgbd,axis=0)# np.vstack(self.obs_rgbd)
        #print(self.obs_rgbd.shape)
        self.obs_state =np.concatenate(self.obs_state,axis=0)# np.vstack(self.obs_state)
        self.actions = np.concatenate(self.actions,axis=0)#np.vstack(self.actions)

    def __len__(self):
        return len(self.obs_rgbd)

    def __getitem__(self, idx):
        action = th.from_numpy(self.actions[idx]).float()
        rgbd = self.obs_rgbd[idx]
        rgbd = rescale_rgbd(rgbd)
        # permute data so that channels are the first dimension as PyTorch expects this
        rgbd = th.from_numpy(rgbd).float().permute((2, 0, 1))
        state = th.from_numpy(self.obs_state[idx]).float()
        return dict(rgbd=rgbd, state=state), action
    

In [4]:
demopath = 'stackcube_500.pkl'
dataset = ManiSkill2Dataset(demopath)
dataloader = DataLoader(dataset, batch_size=100, num_workers=2, pin_memory=True, drop_last=True, shuffle=True)
obs, action = dataset[0]

load in again


  0%|          | 0/500 [00:00<?, ?it/s]

These two using state in training phase

In [5]:
class NatureCNN(nn.Module):
    def __init__(self, image_size=(128, 128), in_channels=8, state_size=32):
        super().__init__()

        extractors = {}

        self.out_features = 0
        feature_size = 256

        # here we use a NatureCNN architecture to process images, but any architecture is permissble here
        cnn = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten()
        )

        # to easily figure out the dimensions after flattening, we pass a test tensor
        test_tensor = th.zeros(
            [in_channels, image_size[0], image_size[1]]
        )
        with th.no_grad():
            n_flatten = cnn(test_tensor[None]).shape[1]
            fc = nn.Sequential(nn.Linear(n_flatten, feature_size), nn.ReLU())
        extractors["rgbd"] = nn.Sequential(cnn, fc)
        self.out_features += feature_size   #image feature size would be 256
        
        # for state data we simply pass it through a single linear layer
        extractors["state"] = nn.Linear(state_size, 64)
        self.out_features += 64

        self.extractors = nn.ModuleDict(extractors)

    def forward(self, observations) -> th.Tensor:
        encoded_tensor_list = []
        # self.extractors contain nn.Modules that do all the processing.
        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))
        return th.cat(encoded_tensor_list, dim=1)

In [6]:
class Policy(nn.Module):
    def __init__(
        self,
        image_size=(128, 128),
        in_channels=8,
        state_size=42,
        hidden_units=[128, 128],
        act_dims=8,
        activation=nn.ReLU,
    ):
        super().__init__()
        self.feature_extractor = NatureCNN(image_size, in_channels, state_size)
        mlp_layers = []
        prev_units = self.feature_extractor.out_features
        for h in hidden_units:
            mlp_layers += [nn.Linear(prev_units, h), activation()]
            prev_units = h
        mlp_layers += [nn.Linear(prev_units, act_dims), nn.Tanh()]
        self.mlp = nn.Sequential(*mlp_layers)

    def forward(self, observations) -> th.Tensor:
        features = self.feature_extractor(observations)
        return self.mlp(features)
'''
# create our policy
obs, action = dataset[0]
rgbd_shape = obs['rgbd'].shape
print(obs['rgbd'].shape)
th.manual_seed(0)
policy = Policy(image_size=rgbd_shape[1:], in_channels=rgbd_shape[0], state_size=obs['state'].shape[0], 
                act_dims=action.shape[0], hidden_units=[256, 256, 256])
# move model to gpu if possible
device = "cuda" if th.cuda.is_available() else "cpu"
policy = policy.to(device)
print(policy)
'''

'\n# create our policy\nobs, action = dataset[0]\nrgbd_shape = obs[\'rgbd\'].shape\nprint(obs[\'rgbd\'].shape)\nth.manual_seed(0)\npolicy = Policy(image_size=rgbd_shape[1:], in_channels=rgbd_shape[0], state_size=obs[\'state\'].shape[0], \n                act_dims=action.shape[0], hidden_units=[256, 256, 256])\n# move model to gpu if possible\ndevice = "cuda" if th.cuda.is_available() else "cpu"\npolicy = policy.to(device)\nprint(policy)\n'

These new network is using without state info, pure rgbd data 

In [7]:
class PureCNN(nn.Module):
    def __init__(self, image_size=(128, 128), in_channels=8):
        super().__init__()

        extractors = {}

        self.out_features = 0
        feature_size = 256

        # here we use a NatureCNN architecture to process images, but any architecture is permissble here
        cnn = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten()
        )

        # to easily figure out the dimensions after flattening, we pass a test tensor
        test_tensor = th.zeros(
            [in_channels, image_size[0], image_size[1]]
        )
        with th.no_grad():
            n_flatten = cnn(test_tensor[None]).shape[1]
            fc = nn.Sequential(nn.Linear(n_flatten, feature_size), nn.ReLU())
        extractors["rgbd"] = nn.Sequential(cnn, fc)
        self.out_features += feature_size   #image feature size would be 256
        
        # for state data we simply pass it through a single linear layer
        #extractors["state"] = nn.Linear(state_size, 64)
        #self.out_features += 64

        self.extractors = nn.ModuleDict(extractors)

    def forward(self, observations) -> th.Tensor:
        encoded_tensor_list = []
        # self.extractors contain nn.Modules that do all the processing.
        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))
        return th.cat(encoded_tensor_list, dim=1)

In [8]:
class VisPolicy(nn.Module):
    def __init__(
        self,
        image_size=(128, 128),
        in_channels=8,
        hidden_units=[128, 128],
        act_dims=8,
        activation=nn.ReLU,
    ):
        super().__init__()
        self.feature_extractor = PureCNN(image_size, in_channels)
        mlp_layers = []
        prev_units = self.feature_extractor.out_features
        for h in hidden_units:
            mlp_layers += [nn.Linear(prev_units, h), activation()]
            prev_units = h
        mlp_layers += [nn.Linear(prev_units, act_dims), nn.Tanh()]
        self.mlp = nn.Sequential(*mlp_layers)

    def forward(self, observations) -> th.Tensor:
        features = self.feature_extractor(observations)
        return self.mlp(features)

# create our policy
obs, action = dataset[0]
rgbd_shape = obs['rgbd'].shape
print(obs['state'].shape)
th.manual_seed(0)
policy = VisPolicy(image_size=rgbd_shape[1:], in_channels=rgbd_shape[0],
                act_dims=action.shape[0], hidden_units=[256, 256, 256])
# move model to gpu if possible
device = "cuda" if th.cuda.is_available() else "cpu"
policy = policy.to(device)
print(policy)

torch.Size([55])
VisPolicy(
  (feature_extractor): PureCNN(
    (extractors): ModuleDict(
      (rgbd): Sequential(
        (0): Sequential(
          (0): Conv2d(8, 32, kernel_size=(8, 8), stride=(4, 4))
          (1): ReLU()
          (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
          (3): ReLU()
          (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
          (5): ReLU()
          (6): Flatten(start_dim=1, end_dim=-1)
        )
        (1): Sequential(
          (0): Linear(in_features=9216, out_features=256, bias=True)
          (1): ReLU()
        )
      )
    )
  )
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=7, bias=True)
    (7): Tanh()
  )
)


### 2.4 Setting up Training, Dataloader, and Logging

With a policy and dataset, we can now write some utility functions to perform a training step, load data in batches, and log results to tensorboard.

In [9]:
loss_fn = nn.MSELoss()

# a short save function to save our model
def save_model(policy, path):
    save_data = dict(
        policy=policy.state_dict(),
    )
    base = os.path.join('./model',path)
    th.save(save_data, base)

def train_step(policy, obs, actions, optim, loss_fn):
    optim.zero_grad()
    # move data to appropriate device first
    obs_device = dict()
    for k in obs:
        obs_device[k] = obs[k].to(device)
    actions = actions.to(device)

    pred_actions = policy(obs_device)
    
    # compute loss and optimize
    loss = loss_fn(actions, pred_actions)
    loss.backward()
    optim.step()
    return loss.item()

Below sets up the logging tools as well which can be viewed with `tensorboard --logdir logs`. You can also open up Tensorboard directly in this notebook

In [10]:
from torch.utils.tensorboard import SummaryWriter

#writer = SummaryWriter(os.path.join('/content/drive/MyDrive/Colab_Notebooks/final/liftcube',f"logs/rgbd_{env_id}"))
writer = SummaryWriter(f"./logs/rgbd_{env_id}_dagger")

### 2.5 Training

We can now create a optimizer and training loop and begin training. The code below will optimize for `iterations = 8000` number of gradient steps at a learning rate of `1e-3`. These parameters are tuned for training on the LiftCube environment and will train a succesful policy that doesn't overfit too much to the dataset. Training time takes around 5-25 minutes depending on hardware.

Note that this is a simple tutorial with a barebones training setup. It doesn't include using a validation dataset, computing success rate during training, regularization or normalization etc.

See final thoughts in section 3 for some simple suggestions on how improve the IL approach.

In [11]:
'''
iterations = 5000
optim = th.optim.Adam(policy.parameters(), lr=1e-3)
best_epoch_loss = np.inf
pbar = tqdm(dataloader, total=iterations)
epoch = 0
steps = 0
while steps < iterations:
    epoch_loss = 0
    for batch in dataloader:
        steps += 1
        obs, actions = batch
        loss_val = train_step(policy, obs, actions, optim, loss_fn)

        # track the loss and print it
        writer.add_scalar("train/mse_loss", loss_val, steps)
        epoch_loss += loss_val
        pbar.set_postfix(dict(loss=loss_val))
        pbar.update(1)

        # periodically save the policy
        if steps % 1000 == 0: save_model(policy, f"ckpt_{steps}_stack.pt")
        if steps >= iterations: break
    
    epoch_loss = epoch_loss / len(dataloader)

    # save a new model if the average MSE loss in an epoch has improved
    if epoch_loss < best_epoch_loss:
        best_epoch_loss = epoch_loss
        save_model(policy, "ckpt_best1_stack.pt")
    
    writer.add_scalar("train/mse_loss_epoch", epoch_loss, epoch)
    epoch += 1
save_model(policy, "ckpt_latest_stack.pt")
'''

'\niterations = 5000\noptim = th.optim.Adam(policy.parameters(), lr=1e-3)\nbest_epoch_loss = np.inf\npbar = tqdm(dataloader, total=iterations)\nepoch = 0\nsteps = 0\nwhile steps < iterations:\n    epoch_loss = 0\n    for batch in dataloader:\n        steps += 1\n        obs, actions = batch\n        loss_val = train_step(policy, obs, actions, optim, loss_fn)\n\n        # track the loss and print it\n        writer.add_scalar("train/mse_loss", loss_val, steps)\n        epoch_loss += loss_val\n        pbar.set_postfix(dict(loss=loss_val))\n        pbar.update(1)\n\n        # periodically save the policy\n        if steps % 1000 == 0: save_model(policy, f"ckpt_{steps}_stack.pt")\n        if steps >= iterations: break\n    \n    epoch_loss = epoch_loss / len(dataloader)\n\n    # save a new model if the average MSE loss in an epoch has improved\n    if epoch_loss < best_epoch_loss:\n        best_epoch_loss = epoch_loss\n        save_model(policy, "ckpt_best1_stack.pt")\n    \n    writer.add

### 2.6 Evaluation

With a trained policy on our hands, we can now create an evaluation environment to compute the success rate and watch the videos. The default settings should train a policy that achieves around 30% success rate

In [12]:
# optionally load a checkpoint
'''
path = os.path.join('/content/drive/MyDrive/Colab_Notebooks/final/',"ckpt_best1_stack.pt")
policy.load_state_dict(th.load(path)["policy"])
'''

'\npath = os.path.join(\'/content/drive/MyDrive/Colab_Notebooks/final/\',"ckpt_best1_stack.pt")\npolicy.load_state_dict(th.load(path)["policy"])\n'

In [13]:
from mani_skill2.utils.wrappers import RecordEpisode
'''
env_id="StackCube-v1"
obs_mode = "rgbd"
control_mode = "pd_ee_delta_pose"
env = gym.make(env_id, obs_mode=obs_mode, control_mode=control_mode)
# RecordEpisode wrapper auto records a new video once an episode is completed
env = RecordEpisode(env, output_dir=f"logs/rgbd_{env_id}/videos")
obs = env.reset(seed=42)

successes = []
num_episodes = 100
i = 0
pbar = tqdm(total=num_episodes)
while i < num_episodes:
    # convert observation to our desired shape and move to appropriate device
    obs = convert_observation(obs)
    obs_device = dict()
    obs['rgbd'] = rescale_rgbd(obs['rgbd'])
    # unsqueeze adds an extra batch dimension and we permute rgbd since PyTorch expects the channel dimension to be first
    obs_device['rgbd'] = th.from_numpy(obs['rgbd']).float().permute(2,0,1).unsqueeze(0).to(device)
    obs_device['state'] = th.from_numpy(obs['state']).float().unsqueeze(0).to(device)
    #print(obs['state'])
    with th.no_grad():
        action = policy(obs_device).cpu().numpy()[0]
    obs, reward, done, info = env.step(action)

    if done:
        successes.append(info['success'])
        obs = env.reset()
        i += 1
        pbar.update(1)
print("Success Rate:", np.mean(successes))
print(successes)
writer.add_scalar("Success Rate",  np.mean(successes), num_episodes)
'''

'\nenv_id="StackCube-v1"\nobs_mode = "rgbd"\ncontrol_mode = "pd_ee_delta_pose"\nenv = gym.make(env_id, obs_mode=obs_mode, control_mode=control_mode)\n# RecordEpisode wrapper auto records a new video once an episode is completed\nenv = RecordEpisode(env, output_dir=f"logs/rgbd_{env_id}/videos")\nobs = env.reset(seed=42)\n\nsuccesses = []\nnum_episodes = 100\ni = 0\npbar = tqdm(total=num_episodes)\nwhile i < num_episodes:\n    # convert observation to our desired shape and move to appropriate device\n    obs = convert_observation(obs)\n    obs_device = dict()\n    obs[\'rgbd\'] = rescale_rgbd(obs[\'rgbd\'])\n    # unsqueeze adds an extra batch dimension and we permute rgbd since PyTorch expects the channel dimension to be first\n    obs_device[\'rgbd\'] = th.from_numpy(obs[\'rgbd\']).float().permute(2,0,1).unsqueeze(0).to(device)\n    obs_device[\'state\'] = th.from_numpy(obs[\'state\']).float().unsqueeze(0).to(device)\n    #print(obs[\'state\'])\n    with th.no_grad():\n        action =

In [14]:
class DAggerdataset(Dataset):
    def __init__(self,data):
       
        self.obs_state = []
        self.obs_rgbd = []
        self.actions = []
        
        for _,v in data.items():    #each k is 'traj_1" ,v would be a dict
            
            
            obs = convert_observation(v["obs"])
            self.obs_rgbd.append(obs['rgbd'][:-1])
            self.obs_state.append(obs['state'][:-1])
            self.actions.append(v["actions"])
        
        self.obs_rgbd = np.concatenate(self.obs_rgbd,axis=0)# np.vstack(self.obs_rgbd)
        
        self.obs_state =np.concatenate(self.obs_state,axis=0)# np.vstack(self.obs_state)
        self.actions = np.concatenate(self.actions,axis=0)#np.vstack(self.actions)



    def __len__(self):
        return len(self.obs_rgbd)
    def __getitem__(self,idx):

        action = th.from_numpy(self.actions[idx]).float()
        rgbd = self.obs_rgbd[idx]
        rgbd = rescale_rgbd(rgbd)
        # permute data so that channels are the first dimension as PyTorch expects this
        rgbd = th.from_numpy(rgbd).float().permute((2, 0, 1))
        state = th.from_numpy(self.obs_state[idx]).float()
        return dict(rgbd=rgbd, state=state), action


In [15]:
def trainBC(dataloader,iterations,policy,loss_fn,writer):
    
    optim = th.optim.Adam(policy.parameters(), lr=1e-3)
    best_epoch_loss = np.inf
    pbar = tqdm(dataloader, total=iterations)
    epoch = 0
    steps = 0
    while steps < iterations:
        epoch_loss = 0
        for batch in dataloader:
            
            steps += 1
            obs, actions = batch
            loss_val = train_step(policy, obs, actions, optim, loss_fn)

            # track the loss and print it
            writer.add_scalar("train/mse_loss", loss_val, steps)
            epoch_loss += loss_val
            pbar.set_postfix(dict(loss=loss_val))
            pbar.update(1)

            # periodically save the policy
            if steps % 1000 == 0: save_model(policy, f"stack/ckpt_{steps}.pt")
            if steps >= iterations: break
        
        epoch_loss = epoch_loss / len(dataloader)

        # save a new model if the average MSE loss in an epoch has improved
        if epoch_loss < best_epoch_loss:
            best_epoch_loss = epoch_loss
            save_model(policy, "stack/ckpt_best.pt")
        
        writer.add_scalar("train/mse_loss_epoch", epoch_loss, epoch)
        epoch += 1
    save_model(policy, "stack/ckpt_latest.pt")
    return policy

In [16]:
from stable_baselines3 import SAC
from tqdm import trange
from trajectory import CollectTrajectories

env_id="StackCube-v1"
obs_mode = "rgbd"
control_mode = "pd_ee_delta_pose"
env = gym.make(env_id, obs_mode=obs_mode, control_mode=control_mode)
obs = env.reset()

#print(obs['extra'].keys())
expert = SAC.load('./logs/stackcube_latest_model.zip',device=device)

def dagger(dictdata,expert,env,policy,writer):       
    #expert,student would be two policy
    
    num_rollout = 20
    loss_fn =  nn.MSELoss()
    for i in tqdm(range(100)):   #100 new rounds/traj
        traj_dataset = DAggerdataset(dictdata)
        new_traj = DataLoader(traj_dataset, batch_size=100, num_workers=16, pin_memory=True, drop_last=True, shuffle=True)
        
        new_obs = {}
        
        reward_per_round = []
        
        max_steps = 5000
        print(f'Rollout {i}, Number of total traj: {len(new_traj)}, Number of total traj from dictdata {len(dictdata.keys())}')
        student = trainBC(dataloader =new_traj,policy=policy,loss_fn = loss_fn,writer=writer,iterations=max_steps)
        print('Finish training student..')
        for i in range(num_rollout):
            
            obs = env.reset()
            done = False
            totalr = 0.
            steps = 0
            obstemp = convert_observation(obs)
            preact = th.from_numpy(obstemp['state']).float().unsqueeze(0)
            expert_action,_ = expert.predict(preact)
            traj_collect = CollectTrajectories(obs, expert_action,env)
            
            while not done:
                #move data to GPU
                obstemp = convert_observation(obs)
                obs_device = dict()
                obs['rgbd'] = rescale_rgbd(obstemp['rgbd'])
                # unsqueeze adds an extra batch dimension and we permute rgbd since PyTorch expects the channel dimension to be first
                obs_device['rgbd'] = th.from_numpy(obstemp['rgbd']).float().permute(2,0,1).unsqueeze(0).to(device)
                obs_device['state'] = th.from_numpy(obstemp['state']).float().unsqueeze(0)
                

                expert_action,_ = expert.predict(obs_device['state'])     #expert's action under this observation, state-based obs
                pred_action = student(obs_device).detach().cpu().numpy()[0]       #image-based obs 
                
                traj_collect.update_extras(obs,expert_action,done)


                obs, r, done, _ = env.step(pred_action)

                totalr += r
                steps += 1
               
                if steps > max_steps:
                    break
            reward_per_round.append(totalr)
            
            
        

        print('returns', reward_per_round)
        print('mean return', np.mean(reward_per_round))
        print('std of return', np.std(reward_per_round))
       
        dictdata[f'traj_{i+100}'] = traj_collect.trajectory()
        
    return new_traj
testdata = np.load('./stackcube_500.pkl',allow_pickle=True)
#print(testdata['traj_0']['obs'].keys())
#dataload = DAggerdataset(testdata)
dagger(testdata,expert,env,policy,writer)
        



  0%|          | 0/100 [00:00<?, ?it/s]

Rollout 0, Number of total traj: 108, Number of total traj from dictdata 500


  0%|          | 0/5000 [00:00<?, ?it/s]

Finish training student..


AttributeError: 'CollectTrajectories' object has no attribute 'tcp_pose'